<a href="https://colab.research.google.com/github/GabiiKatherin/Analise_Corinthians_Brasileirao/blob/main/Analise_de_dados_Corinthians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Instalando bibliotecas:

In [2]:
#instalando bibliotecas:
!pip install beautifulsoup4

import requests
#O BeautifulSoup é pra analisar e extrair dados do HTML ou XML
from bs4 import BeautifulSoup
import pandas as pd

2. Webscraping e consolidação dos dados:

In [14]:
#definindo as urls com os dados:
url_gols = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista'
url_cartoes = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista/cartoes'
url_rendimento = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista/rendimento'

In [4]:
#requisicao HTTP
response = requests.get(url_gols)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#procurando a tabela pela tag <table> - me certifiquei disso inspecionando o elemento na página
tables = soup.find_all('table')

In [6]:
#converte a tabela num Dataframe:
#a função read_html é do Pandas e basicamente converte o conteúdo HTML da tala num Dataframe.
df_list = []
for table in tables:
    df = pd.read_html(str(table))[0]
    df_list.append(df)

In [ ]:
print(df_list[0])

Essa etapa acusou um erro, dizendo que não havia nada na minha tabela, provavelmente porque os dados não foram capturados como eu gostaria. Então, fui testar e fazer uma exibição do que estava sendo capturado e identificar o problema.

In [ ]:
#verificar o que foi capturado:
print(soup.prettify()[:1000])  #top 1000

Isso aqui é interessante! O que aconteceu foi que a minha requisição foi bloqueada pelo servidor. Provavelmente, porque ele está evitando acessos automatizados por scripts de scraping, uma medida de segurança super válida, inclusive.

Decidi testar mudar o User-Agent e em vez de usar o request padrão, imitar um navegador real.

In [24]:
#Define um cabeçalho HTTP para simular uma requisição feita por um navegador.
#O User-Agent especifica as informações do navegador e do sistema operacional
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#Faz uma requisição GET na url que eu defini antes
response_g = requests.get(url_gols, headers=headers)
response_c = requests.get(url_cartoes, headers=headers)
response_r = requests.get(url_rendimento, headers=headers)

#Converte o HTML num soup - facilita a manipulação e extração dos dados.
soup_g = BeautifulSoup(response_g.text, 'html.parser')
soup_c = BeautifulSoup(response_c.text, 'html.parser')
soup_r = BeautifulSoup(response_r.text, 'html.parser')

In [53]:
#encontrando minhas tabelas:
tables = soup_g.find_all('table')
print(f"Total de tabelas encontradas: {len(tables)}")

Total de tabelas encontradas: 2


Como eu tenho mais de uma tabela, vou precisar de uma estrutura de repetição para capturar todas elas.

In [34]:
for t, table in enumerate(tables):
    df_gols = pd.read_html(str(table))[0]
    print(f"Tabela {t+1}:")
    print(df_gols)

Tabela 1:
     POS              Nome   J  G
0    1.0     Rodrigo Garro  25  5
1    2.0      Yuri Alberto  20  4
2    NaN      Ángel Romero  19  4
3    4.0              Cacá  21  3
4    5.0    Wesley Ribeiro  23  2
5    NaN      Talles Magno   6  2
6    7.0     Igor Coronado  22  1
7    NaN      Matheus Bidu  12  1
8    NaN           Giovane  11  1
9    NaN    Pedro Henrique  10  1
10  11.0           Raniele  23  0
11   NaN              Hugo  22  0
12   NaN        Mateuzinho  22  0
13   NaN       Breno Bidon  18  0
14   NaN      Félix Torres  17  0
15   NaN        Pedro Raúl  17  0
16   NaN            Fagner  12  0
17   NaN        Hugo Souza  11  0
18   NaN              Ryan  11  0
19   NaN  Gustavo Henrique  10  0
20   NaN     André Ramalho   9  0
21   NaN   Matheus Donelli   7  0
22   NaN     Carlos Miguel   6  0
23   NaN           Charles   6  0
Tabela 2:
     POS              Nome   J  A
0    1.0     Rodrigo Garro  25  4
1    NaN              Hugo  22  4
2    3.0      Yuri Alberto  

<ipython-input-34-a67a650f79a4>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_gols = pd.read_html(str(table))[0]
<ipython-input-34-a67a650f79a4>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_gols = pd.read_html(str(table))[0]


Como as tabelas são muito parecidas, eu vou associá-las incluindo as colunas de Gols e Assistência num DataFrame consolidado. Pra isso, eu começo executando novamente a chamada do DataFrame, mas desta vez, criando uma variável temporária para armazenar essas tabelas.

In [62]:
#Crio as variáveis:
df_gols = pd.DataFrame()
df_assistencia = pd.DataFrame()

#Eu chamo meu for novamente
for t, table in enumerate(soup_g.find_all('table')):
    df_temp = pd.read_html(str(table))[0]
#Porém, eu atribuo à essa tabela criada anteriormente às tabelas geradas nos ciclos dos loops t==0 e t==1
    if t == 0:
        df_gols = df_temp  # Armazenar a tabela de gols
    elif t == 1:
        df_assistencia = df_temp  # Armazenar a tabela de assistências

    # Exibir a tabela temporária
    print(f"Tabela {t+1}:")
    print(df_temp)

Tabela 1:
     POS              Nome   J  G
0    1.0     Rodrigo Garro  25  5
1    2.0      Yuri Alberto  20  4
2    NaN      Ángel Romero  19  4
3    4.0              Cacá  21  3
4    5.0    Wesley Ribeiro  23  2
5    NaN      Talles Magno   6  2
6    7.0     Igor Coronado  22  1
7    NaN      Matheus Bidu  12  1
8    NaN           Giovane  11  1
9    NaN    Pedro Henrique  10  1
10  11.0           Raniele  23  0
11   NaN              Hugo  22  0
12   NaN        Mateuzinho  22  0
13   NaN       Breno Bidon  18  0
14   NaN      Félix Torres  17  0
15   NaN        Pedro Raúl  17  0
16   NaN            Fagner  12  0
17   NaN        Hugo Souza  11  0
18   NaN              Ryan  11  0
19   NaN  Gustavo Henrique  10  0
20   NaN     André Ramalho   9  0
21   NaN   Matheus Donelli   7  0
22   NaN     Carlos Miguel   6  0
23   NaN           Charles   6  0
Tabela 2:
     POS              Nome   J  A
0    1.0     Rodrigo Garro  25  4
1    NaN              Hugo  22  4
2    3.0      Yuri Alberto  

<ipython-input-62-817a6b2665e9>:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_temp = pd.read_html(str(table))[0]
<ipython-input-62-817a6b2665e9>:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_temp = pd.read_html(str(table))[0]


Em seguida, eu de fato uso o pd.merge para unificar os dados e deixar o DataFrame como eu gostaria de manipulá-lo.

In [89]:
#pd.merge para unificar:
df_unificado = pd.merge(df_gols, df_assistencia, on=['Nome'], how='outer', suffixes=('_gols', '_assist'))
pd.set_option('display.float_format', '{:.0f}'.format)

#Substituo NaN por zero.
df_unificado.fillna(0, inplace=True)
#Personalizo as colunas:
df_unificado = df_unificado[['POS_gols', 'Nome', 'J_assist','J_gols', 'A', 'G']]

# Exibir o DataFrame unificado
print("Tabela Unificada:")
print(df_unificado)

Tabela Unificada:
    POS_gols              Nome  J_assist  J_gols  A  G
0          1     Rodrigo Garro        25      25  4  5
1          2      Yuri Alberto        20      20  3  4
2          0      Ángel Romero        19      19  0  4
3          4              Cacá        21      21  2  3
4          5    Wesley Ribeiro        23      23  1  2
5          0      Talles Magno         0       6  0  2
6          7     Igor Coronado        22      22  2  1
7          0      Matheus Bidu        12      12  1  1
8          0           Giovane        11      11  0  1
9          0    Pedro Henrique        10      10  1  1
10        11           Raniele        23      23  0  0
11         0              Hugo        22      22  4  0
12         0        Mateuzinho        22      22  0  0
13         0       Breno Bidon        18      18  0  0
14         0      Félix Torres        17      17  0  0
15         0        Pedro Raúl        17      17  0  0
16         0            Fagner        12      1

In [90]:
tables = soup_c.find_all('table')

df_cartoes = pd.read_html(str(tables[0]))[0]
print("Tabela de Cartoes:")
print(df_cartoes)

Tabela de Cartoes:
    POS              Nome   J  CA  CV  pts
0     1  Gustavo Henrique  10   5   1    8
1     2           Raniele  23   6   0    6
2   NaN              Cacá  21   3   1    6
3     4      Alex Santana   8   4   0    4
4   NaN     Igor Coronado  22   4   0    4
5   NaN           Caetano   5   1   1    4
6     7           Charles   6   3   0    3
7   NaN      Félix Torres  17   3   0    3
8   NaN     André Ramalho   9   3   0    3
9    10              Hugo  22   2   0    2
10  NaN        Mateuzinho  22   2   0    2
11  NaN        Hugo Souza  11   2   0    2
12   13            Maycon   2   1   0    1
13  NaN          Léo Mana   5   1   0    1
14  NaN      Matheus Bidu  12   1   0    1
15  NaN            Fagner  12   1   0    1
16   17     Renato Santos   0   0   0    0
17  NaN    Diego Palacios   0   0   0    0
18  NaN      Felipe Longo   0   0   0    0
19  NaN   Matheus Donelli   7   0   0    0


<ipython-input-90-be9983ee573f>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_cartoes = pd.read_html(str(tables[0]))[0]


In [93]:
#pd.merge para unificar:
df_unificado = pd.merge(df_unificado, df_cartoes, on=['Nome'], how='outer', suffixes=('', '_cartoes'))

pd.set_option('display.float_format', '{:.0f}'.format)
df_unificado.fillna(0, inplace=True)

df_unificado = df_unificado[['POS_gols', 'Nome', 'J_assist','A','J_gols','G','J_cartoes','CA','CV', 'pts']]
df_jogadores = df_unificado
print(df_jogadores)

    POS_gols              Nome  J_assist  A  J_gols  G  J_cartoes  CA  CV  pts
0          1     Rodrigo Garro        25  4      25  5          0   0   0    0
1          2      Yuri Alberto        20  3      20  4          0   0   0    0
2          0      Ángel Romero        19  0      19  4          0   0   0    0
3          4              Cacá        21  2      21  3         21   3   1    6
4          5    Wesley Ribeiro        23  1      23  2          0   0   0    0
5          0      Talles Magno         0  0       6  2          0   0   0    0
6          7     Igor Coronado        22  2      22  1         22   4   0    4
7          0      Matheus Bidu        12  1      12  1         12   1   0    1
8          0           Giovane        11  0      11  1          0   0   0    0
9          0    Pedro Henrique        10  1      10  1          0   0   0    0
10        11           Raniele        23  0      23  0         23   6   0    6
11         0              Hugo        22  4      22 

In [44]:
#encontrando minhas tabelas:
tables = soup_r.find_all('table')
print(f"Total de tabelas encontradas: {len(tables)}")

Total de tabelas encontradas: 3


In [45]:
for t, table in enumerate(tables):
    df_rendimento = pd.read_html(str(table))[0]
    print(f"Tabela {t+1}:")
    print(df_rendimento)

Tabela 1:
           Categoria  Gols                               Jogo          Datas
0       Gols pró (M)     3   CORCorinthians3 - 0FLUFluminense  dom., 28 abr.
1       Gols pró (V)     2  AGOAtlético-GO2 - 2CORCorinthians  ter., 11 jun.
2    Gols contra (M)     2    CORCorinthians2 - 2SAOSão Paulo  dom., 16 jun.
3    Gols contra (V)     3     CRUCruzeiro3 - 0CORCorinthians   dom., 7 jul.
4  Margem de Vitória     3   CORCorinthians3 - 0FLUFluminense  dom., 28 abr.
5  Margem de Derrota     3     CRUCruzeiro3 - 0CORCorinthians   dom., 7 jul.
Tabela 2:
                       Categoria  Partidas
0    Maior sequência de vitórias         2
1    Atual sequência de vitórias         1
2        Maior sequência invicta         3
3        Atual sequência invicta         1
4    Maior sequência de derrotas         2
5    Atual sequência de derrotas         0
6   Maior sequência sem vitórias         9
7  Actual sequência sem vitórias         0
Tabela 3:
          Categoria      PT                 

<ipython-input-45-3d1d8d0a6f26>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_rendimento = pd.read_html(str(table))[0]
<ipython-input-45-3d1d8d0a6f26>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_rendimento = pd.read_html(str(table))[0]
<ipython-input-45-3d1d8d0a6f26>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_rendimento = pd.read_html(str(table))[0]


Show! Agora eu tenho um DataFrame com o consolidado por jogador. Fiz o webscraping, consolidei os dados e tratei o que precisava. Agora, vou analisar as informações que obtive: